<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Intro to SQL: Connecting to Databases

_Authors: Dave Yerrington (SF), Fancesco Mosconi (SF)_

---

### Learning Objectives
*After this lesson, you will be able to:*
- Connect to a local or remote database using the command line, Python, or `pandas`.
- Connect to a local or remote database using SQLite Manager (for SQLite) or Postico (for POSTGRES).
- Perform queries using SELECT.
- Perform simple aggregations suing COUNT and MAX/MIN/SUM.

### Lesson Guide
- [Connecting to a Local Database](#connecting-to-localdb)
    - [SQLite](#sqlite)
    - [Interacting with SQLite](#interacting-sqlite)
    - [Common SQL Command Patterns](#command-patterns)
    - [SQLite Command Line Utility](#sqlite-command-line)
    - [Creating Tables and Adding Columns](#sqlite-tables-columns)
    - [Adding Data](#adding-data)
    - [Updating Records](#updating-records)
    - [Removing Records](#removing-records)
- [Connecting to a Remote Database](#remote-database)
    - [PostgreSQL](#postgresql)
- [Comparison of SQL Commands](#comparison-commands)
- [SQLite from Python](#python-sqlite)
    - [SQLite3 Package](#sqlite3)
    - [SQLite3: Adding Data](#sqlite3-adding)
    - [Adding Data From a `.csv` File](#adding-csv)
- [`pandas` Connector](#pandas-connector)
    - [Writing Data Into a Database](#pandas-writing)
    - [Reading Data From a Database](#pandas-reading)
- [SQL Syntax](#sql-syntax)
    - [Aggregations](#aggregations)
- [Independent Practice: Querying a Database](#ind-practice)

<a name="connecting-to-localdb"></a>
## Connecting to a Local Database

---

A database can be local or remote; it can span a single machine or be distributed with replicated data over several. The latter configuration is called _sharding_.

Let's start by connecting to a local _SQLite_ database.

<a id='sqlite'></a>

### SQLite

**[SQLite](https://sqlite.org/)** is a database software package built on the Structured Query Language [(SQL)](https://www.sqlcourse.com/intro.html). It is similar to other SQL databases, such as [PostgreSQL](http://www.postgresql.org/), [MySQL](https://www.mysql.com/), Oracle, and Microsoft SQL Server, except it is *file-based* rather than *server-based*. This makes it easy to set up and use for small projects but less suitable for production environments. Once you are familiar with SQLite, the same ideas and similar syntax can be applied to other SQL databases.

Version 3 of SQLite is bundled with most Python distributions (including our Anaconda distribution). There are two options we recommend for browsing SQLite3 databases:

- [SQLite Browser](http://sqlitebrowser.org/), A free, cross-platform solution.
- [SQLite Manager](https://addons.mozilla.org/en-US/firefox/addon/sqlite-manager/?src), a Firefox add-on for viewing SQLite database files via a simple GUI.


_We'll be using the SQLite Browser in this lesson. If you haven't already, install SQLite Browser._  

    brew cask install sqlitebrowser

<a id='interacting-sqlite'></a>

### Interacting with SQLite

There are multiple ways of interacting with a SQLite database, including:

1. SQLite command line utility
2. Python's `sqlite3` package
3. `pandas`' SQL Interface
4. High-level ORMs (e.g., SQLAlchemy, Django ORM, etc.)

Let's start with the first method. All of these methods provide some form of wrapper, or set of convenience functions, for interacting with SQLite. Behind the scenes, the Structured Query Language (SQL) itself defines the interface to the database software. This underlying SQL syntax will be visible to a greater or lesser degree, depending on the method chosen.

<a id='command-patterns'></a>

### Common SQL Command Patterns

The SQL command set has a rich syntax with numerous options, but most of the commonly used commands follow a few simple patterns. A basic familiarity of these patterns is helpful when working in SQL:

    CREATE TABLE ...
    ALTER TABLE ... ADD COLUMN ...
    INSERT INTO ... VALUES ...
    UPDATE ... SET ... WHERE ...
    SELECT ... FROM ... WHERE ...
    SELECT ... FROM ... JOIN ... ON ...
    DELETE FROM ... WHERE ...

<a id='sqlite-command-line'></a>

### SQLite Command Line Utility

The first method we'll explore is connecting to SQLite via the built-in [command line utility](https://www.sqlite.org/sqlite.html).  

> Note: The commands in this section should be executed within your normal terminal shell.

To start a new session of the interpreter, simply open your terminal and type `sqlite3`, followed by the name of the database file. If the file does not yet exist, SQLite will create it.

    $ sqlite3 test1.sqlite

    SQLite version 3.13.0 2016-05-18 10:57:30
    Enter ".help" for usage hints.
    sqlite>

Notice that your terminal prompt changes to `sqlite>`, indicating that you are now entering commands into the SQLite command line utility. Take a quick look at the `help` command:

    sqlite> .help
Display the current databases — you should see the new file `test1.db`:

    sqlite> .databases
    
In order to execute a SQL statement, you will need to use the semicolon syntax (';') to designate the end of a statement. The statement below used to appear when initiating older versions of SQLite3 in terminal.
    
    "Enter SQL statements terminated with a ';'."
 

<a id='sqlite-tables-columns'></a>

### Creating Tables and Adding Columns

Create a table called `table1` with a single column, `field1`, containing an INTEGER PRIMARY KEY.

    sqlite> CREATE TABLE table1 (field1 INTEGER PRIMARY KEY);

Add a few more columns to `table1`.

    sqlite> ALTER TABLE table1 ADD COLUMN field2 VARCHAR(16);
    sqlite> ALTER TABLE table1 ADD COLUMN field3 REAL;
    sqlite> ALTER TABLE table1 ADD COLUMN field4 TEXT;

Notice the different field types in the ALTER TABLE commands. SQLite supports several different [field types](https://www.sqlite.org/datatype3.html), including INTEGERS, variable length character fields, or VARCHAR (with a max length), TEXT fields, and 'REALS,' which are used to store floating point numbers.

Next, verify that the table was created.

    sqlite> .tables

You can check the schema of the table using `.schema`, which shows the commands that would be necessary to create the database tables from scratch.  

    sqlite> .schema

Notice that, in this case, our `table1` could have been created with a single command rather than by adding each column individually.

<a id='adding-data'></a>

### Adding data

Let's add some data:

    sqlite> INSERT INTO table1 VALUES (1, 'Henry James', 42, '75 Mission Street, San Francisco, CA');
    sqlite> INSERT INTO table1 VALUES (2, 'Carol James', 40, '75 Mission Street, San Francisco, CA');
    sqlite> INSERT INTO table1 VALUES (3, 'Jesse James', 12, '75 Mission Street, San Francisco, CA');

Notice that the first column has unique values — this is a requirement for the PRIMARY KEY (PK) column. If we try to add a record using an existing PK value, we'll get an error.

    sqlite> INSERT INTO table1 VALUES (3, 'Julie James', 10, '75 Mission Street, San Francisco, CA');
    Error: UNIQUE constraint failed: table1.field1

Fortunately, SQLite has some built-in functionality to auto-increment the PK value — just set the value of the PK field to NULL while running the INSERT command, and it will automatically be set to a valid value.

    sqlite> INSERT INTO table1 VALUES (NULL, 'Julie James', 10, '75 Mission Street, San Francisco, CA');

Now that we have some data, take a look at the database using the **SQLite Browser**.

1. Open the app.
2. Open SQLite3 file "Open Database" from the file menu. 
   - _This file will be most likely be in the directory you were in when we started using `sqlite3` commands._
3. After connecting to the database, click on "Browse Data" in the tabbed UI.

Notice that the value in `field1` for the "Julie James" record has been automatically set to 4.

<a id='updating-records'></a>

### Updating Records

Suppose we need to update an existing record with new data — e.g., maybe Julie James is 9. For this, we use the UPDATE command.

    sqlite> UPDATE table1 SET field3=9 WHERE field1=4;

<a id='removing-records'></a>
### Removing Records

To remove records, use the DELETE command.

    sqlite> DELETE FROM table1 WHERE field2 like '%Jesse%';

Use SQLiteManager to verify that the "Jesse James" record has been removed. To exit the SQLite interpreter, type `.exit`.

    sqlite>  .exit

<a name="remote-database"></a>
## Connecting to a Remote Database

---

> Instructor Note: You will need to update this section with appropriate AWS configuration settings!

<a id='postgresql'></a>
### PostgreSQL

**[PostgreSQL](http://www.postgresql.org/)** is a powerful SQL-based relational database.

#### PostgreSQL Syntax

GA provides a PostgreSQL database instance at the following address:

You can connect to it using:

> ### OSX Users
> For OSX, we install PostgreSQL via the Postgres.app package from [postgresapp.com](http://postgresapp.com/).
> Also, for easy access, add this alias to your `.bash_profile`:
>
> **Edit the file:** <br>
> `sudo nano ~/.bash_profile`
>
> **Add this line:** <br>
> `alias psql=/Applications/Postgres.app/Contents/Versions/9.5/bin/psql`  
>> _"`control+O`"_ to 'WriteOut'  
>> _"`return/enter`"_ to confirm  
>> _"`control+X`"_ to exit  
>
> **Reload your profile:** <br>
> `source ~/.bash_profile`
>
> ** Connect to GA's PostgreSQL DB**  
> `psql -h dsi.c20gkj5cvu3l.us-east-1.rds.amazonaws.com -p 5432 -U dsi_student titanic`  
    Password: gastudents

PosgreSQL accepts the same syntax as SQLite, with exception of a few system commands. Here are the most common:

- `\q`: Quit/exit.
- `\c __database__`: Connect to a database.
- `\d __table__`: Show table definition, including triggers.
- `\dt *.*`: List tables from all schemas (if `*.*` is omitted, this will only show SEARCH_PATH ones).
- `\l`: List databases.
- `\dn`: List schemas.
- `\df`: List functions.
- `\dv`: List views.
- `\dt`: List tables.
- `\df+ __function`: Show function SQL code.
- `\x`: Pretty-format query results instead displaying not-so-useful ASCII tables.


Suggestions:
- If a command is run with an `-E` flag, it will describe the underlaying queries of the `\` commands (good for learning!).
- Most `\d` commands support additional parameters of `__schema__.name__` and accept wildcards like `*.*`.

<a id='comparison-commands'></a>
## Comparison of SQL Commands

---


|Task|MySQL|PostgreSQL|SQLite|
|---|---|---|---|
|Connect to a database|mysql &lt;dbname&gt;| psql &lt;dbname&gt;|sqlite3 &lt;filename&gt;|
|Client help|help contents|\?|.help|
|SQL help|help contents|\h|n/a|
|List databases|SHOW DATABASES;|\l|.databases|
|Change database|USE &lt;dbname&gt;|\c <dbname&gt;|n/a|
|List tables|SHOW TABLES;|\dt|.tables|
|Show table|info DESCRIBE &lt;tablename&gt;|\d &lt;tablename&gt;|.schema &lt;tablename&gt;|
|Load data|LOAD DATA INFILE &lt;file&gt; |\i &lt;file&gt;|.import &lt;file&gt; &lt;table&gt;|
|Export data|SELECT ... INTO OUTFILE &lt;file&gt;|\o &lt;file&gt;|.dump &lt;table&gt;|
|Exit the client|quit (or exit)| \q|.exit|

<a name="intermission"></a>

## Intermission

---

We have seen how to connect to a local SQLite database and a remote postgreSQL database.

**Check:** What SQL operations have we learned so far?

**Check:** What commands have we learned for SQLite and PostgreSQL?


<a name="python-sqlite"></a>
## Interacting with SQLite from Python

---

<a id='sqlite3'></a>
### The SQLite3 Package

The command line utility can be useful for basic SQL tasks, but, because we're using Python, it is often easier to access SQLite directly through Python code. We can use the Python [SQLite3](https://docs.python.org/2.7/library/sqlite3.html) package for just this purpose.

**Below, we open a connection to a SQLite database file using Python. Like before, if the file does not already exist, it will automatically be created.**

In [ ]:
# import sqlite3
# sqlite_db = '../datasets/test_db.sqlite'
# conn = sqlite3.connect(sqlite_db) 
# c = conn.cursor()

_Note: The `.cursor()` method is necessary because, in many DB structures, a cursor is needed to iterate over the rows of data. The cursor also functions as a means of fetching information when using SQLite3. _

**The syntax to create a table is similar to the console, only now we use the `.execute()` method of the cursor object (`c`) we just created:**

In [ ]:
# c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')

### Save (commit) the changes.
# conn.commit()

With the database saved, the table should now be viewable using SQLiteManager.

<a id='sqlite3-adding'></a>
### SQLite3: Adding Data

Because we're back in Python, we can now use regular programming techniques in conjunction with the SQLite connection.  In particular, the cursor's `.execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.

See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

In [ ]:
# last_sale = (None, 4000, 5, 22, 619000)
# c.execute('INSERT INTO houses VALUES (?,?,?,?,?)', last_sale)

### Remember to commit the changes.
# conn.commit()

Notice that, in this syntax, we use Python's `None` value, rather than `NULL`, to trigger SQLite and auto-increment the Primary Key. 

There is a related cursor method, `.executemany()`, which takes an array of tuples and loops through them, substituting one tuple at a time.

In [ ]:
# recent_sales = [
#   (None, 2390, 4, 34, 319000),
#   (None, 1870, 3, 14, 289000),
#   (None, 1505, 3, 90, 269000),
# ]

# c.executemany('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', recent_sales)

# conn.commit()

Once again, use SQLite Browser to verify the database's contents. 

![SQLite Manager](https://snag.gy/xc8W5M.jpg)

<a id='adding-csv'></a>

### Adding Data From a `.csv` File

Next, let's load our `housing.csv` data into an array and then `INSERT` those records into the database. In this example, we'll use the `numpy` `.genfromtxt()` function to read the file and parse its contents. 

In [ ]:
# from numpy import genfromtxt

### import into nparray of ints, then convert to list of lists
# data = (genfromtxt('datasets/housing-data.csv', dtype='i8', 
#                     delimiter=',', skip_header=1)).tolist()

### append a None value to beginning of each sub-list
# for d in data:
#     d.insert(0, None)

In [ ]:
### Loop through data, running an INSERT on each record (i.e., sublist).
# for d in data:
#     c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

# conn.commit()

> **Remember:** All elements in a `numpy` array must be the same data type, so, if we want to "add a `None`" to each row, we would need to work around this. Lists can contain mixed types, so this is one approach.

Still, in this case, the value we're adding is the same for all records, so we can simply use a `None` in the INSERT statement.

**How do you delete data?**

In [ ]:
### Similar syntax as before.
# results = c.execute("SELECT * FROM houses WHERE bdrms = 4")

### Here, results is a cursor object — use `.fetchall()` to extract a list.
# results.fetchall()

<a id='pandas-connector'></a>

## `pandas` Connector

---

While databases provide many analytical capabilities, it's often useful to pull data back into Python for more flexible programming. Large, fixed operations would be more efficient in a database, but `pandas` allows for interactive processing.

For example, imagine you want to aggregate nightly logins or sales to present a report or dashboard. Most likely, you wouldn't be changing or operating on a large data set. You could perform this aggregation efficiently within a database.

However, if we want to investigate login or sales data further and ask more interactive questions, using Python would be more practical.

In [ ]:
import pandas as pd

`pandas` can connect to most relational databases. In this demonstration, we'll create and connect to a SQLite database.

SQLite creates portable SQL databases saved in a single file. These databases are stored efficiently and allow for  fast querying, making them ideal for small databases or ones that need to be moved across machines.

<a id='pandas-writing'></a>

### Writing Data Into a Database

Data in `pandas` can be loaded into a relational database. For the most part, `pandas` can use column information to infer the schema for the table it creates. In the next demo, we will use the Rossmann Store's data set.

In [ ]:
# data = pd.read_csv('../datasets/housing-data.csv', low_memory=False)  # Low_memory = Gets rid of ambigious warning... nothing to see here
# data.head()

Data is moved to the database through the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes the following arguments:
- `name`, the table name to create.
- `con`, a connection to a database.
- `index`, whether or not to input the index column.
- `schema`, if we want to write a custom schema for the new table.
- `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail.
    
> ### BTW, what was with the `conn` object?  
> We defined it in a previous example:
> ```python
import sqlite3
sqlite_db = './datasets/test_db.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()
```

In [ ]:
# data.to_sql('houses_pandas',             # Name of the table.
#             con=conn,                    # The handle to the file that is set up.
#             if_exists='replace',         # Overwrite, append, or fail.
#             index=False)                 # Add index as column.

<a id='pandas-reading'></a>

### Reading Data From a Database
_Result as a DataFrame_

If we already have data in our database, we can use `pandas` to query it. Querying is done through the `read_sql` command in the `sql` module.

In [ ]:
# df = pd.read_sql('SELECT * FROM houses_pandas LIMIT 10', con=conn)

<a name="sql-syntax"></a>

## SQL Syntax

---

### SQL Operators

> Note: Each of the following can be demonstrated in `pandas` using the data we've set up above. A demo and check are included for each, but we can try out a few before we begin.

#### SELECT

Every query should start with `SELECT`. `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, which identifies the table from which we're retrieving the data.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` returns *all* of the columns.

Housing data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** What do `sqft`, `bdrms`, and `price` refer to? What will they return?
>
```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE

`WHERE`, which follows the `FROM` clause, is used to filter tables using specific criteria.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is a filter that's applied to the rows. The rows that match the condition will be included in the output.

**Check:** Write a query that returns the `sqft`, `bdrms`, and `age` for houses older than 60.

In [ ]:
# A:

<a name="aggregations"></a>

### Aggregations

Aggregations (or aggregate functions) are functions in which the values of multiple rows are grouped together as an input on certain criteria to form a single value of more significant meaning or measurement. Examples are sets, bags, or lists.

Aggregate funtions include:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL, they are performed in a `SELECT` statement like the following:

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

In [ ]:
# A:

<a name="ind-practice"></a>

## Independent Practice: Querying a Database

---

Practice querying the SQLite database we've created using any of the methods you've learned so far:

- Console connection
- Python's `sqlite3` package
- `pandas`
- [SQLite Browser](http://www.sqlitebrowser.org/) (or Firefox's browser extension, [SQLite Manager](https://addons.mozilla.org/en-US/firefox/addon/sqlite-manager/?src))

Practice querying the PostgreSQL database found here [add url](http://) using:
- Console connection
- Python's `sqlite3` package
- `pandas`
- [Postico](https://eggerapps.at/postico/)


### What is the average price per room for one-bedroom apartments?

In [ ]:
# A:

### How old is the oldest three-bedroom apartment?

In [ ]:
# A:

### How old is the youngest apartment?

In [ ]:
# A:

### What is the average age for the whole data set?

In [ ]:
# A:

### What is the average age for each bedroom size?
_Tip: This will most likely require `pandas`/SQL integration._

In [ ]:
# A:

**If you finish, try completing the first sections of [SQL Zoo](http://www.sqlzoo.net).**

<a id='additional-resources'></a>

## Additional Resources 

---

- [SQLite3's website](http://www.sqlite.org)  
- [SQLite — Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL Zoo](http://www.sqlzoo.net)